In [1]:
import gym, pickle, argparse, json, logging
from copy import deepcopy
import ray
from gail.gail import GAILTrainer

from ray import tune
from ray.rllib.agents.ppo.ppo_policy_graph import PPOPolicyGraph
from ray.rllib.agents.ppo.ppo import DEFAULT_CONFIG
from ray.rllib.agents import Trainer
from ray.rllib.evaluation import PolicyEvaluator, SampleBatch, MultiAgentSampleBatchBuilder
from ray.rllib.offline.json_writer import JsonWriter
from ray.rllib.offline.json_reader import JsonReader
from ray.rllib.evaluation.sample_batch import DEFAULT_POLICY_ID
from ray.rllib.evaluation.metrics import collect_metrics
from ray.tune.registry import register_env
from ray.rllib.utils.annotations import override

from flow.utils.registry import make_create_env
from flow.utils.rllib import FlowParamsEncoder, get_flow_params
logger = logging.getLogger(__name__)
ray.init(ignore_reinit_error=True)

2019-05-30 20:28:51,755	INFO node.py:469 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-05-30_20-28-51_38/logs.
2019-05-30 20:28:51,938	INFO services.py:407 -- Waiting for redis server at 127.0.0.1:41375 to respond...
2019-05-30 20:28:52,191	INFO services.py:407 -- Waiting for redis server at 127.0.0.1:49651 to respond...
2019-05-30 20:28:52,195	INFO services.py:804 -- Starting Redis shard with 10.0 GB max memory.
2019-05-30 20:28:52,301	INFO node.py:483 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-05-30_20-28-51_38/logs.
2019-05-30 20:28:52,304	INFO services.py:1427 -- Starting the Plasma object store with 18.23 GB memory using /dev/shm.


{'node_ip_address': '10.138.0.2',
 'object_store_address': '/tmp/ray/session_2019-05-30_20-28-51_38/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2019-05-30_20-28-51_38/sockets/raylet',
 'redis_address': '10.138.0.2:41375',
 'webui_url': None}

In [2]:
num_cpus = 11
num_rollouts = 3 
horizon = 750
gae_lambda = 0.97
step_size = 5e-4
num_iter = 10
benchmark_name = "multi_merge"
exp_name = "latest"

In [3]:
config = deepcopy(DEFAULT_CONFIG)
config["num_workers"] = min(num_cpus, num_rollouts)
config["train_batch_size"] = horizon * num_rollouts
config["sample_batch_size"] = horizon
config["use_gae"] = True
config["horizon"] = horizon
config["lambda"] = gae_lambda
config["lr"] = step_size
config["vf_clip_param"] = 1e6
config["num_sgd_iter"] = 10
config['clip_actions'] = False  # FIXME(ev) temporary ray bug
config["model"]["fcnet_hiddens"] = [128, 64, 32]
config["observation_filter"] = "NoFilter"
config["entropy_coeff"] = 0.0
config["expert_path"] = '/headless/rl_project/flow_codes/InverseRL/expert_sample'
config["discrim_hidden_size"] = 128

benchmark = __import__(
            "flow.benchmarks.%s" % benchmark_name, fromlist=["flow_params"])
flow_params = benchmark.gail_flow_params

# save the flow params for replay
flow_json = json.dumps(
    flow_params, cls=FlowParamsEncoder, sort_keys=True, indent=4)
config['env_config']['flow_params'] = flow_json

In [4]:
create_env, env_name = make_create_env(params=flow_params, version=0)
register_env(env_name, create_env)
env = create_env()

# we don't need this config
POLICY_ID = DEFAULT_POLICY_ID 
default_policy = (PPOPolicyGraph, env.observation_space, env.action_space, {})
policy_graph = {POLICY_ID: default_policy}
config["multiagent"] = {
        'policy_graphs': policy_graph,
        'policy_mapping_fn': tune.function(lambda agent_id: POLICY_ID),
        'policies_to_train': [POLICY_ID]
    }

In [5]:
agent = GAILTrainer(config, env_name)

2019-05-30 20:29:01,504	WARNING json_reader.py:52 -- Treating input directory as glob pattern: /headless/rl_project/flow_codes/InverseRL/expert_sample/*.json
2019-05-30 20:29:01,511	INFO json_reader.py:65 -- Found 1 input files.
2019-05-30 20:29:02,968	INFO policy_evaluator.py:311 -- Creating policy evaluation worker 0 on CPU (please ignore any CUDA init errors)
/opt/conda/envs/flow-latest/lib/python3.5/site-packages/tensorflow/python/ops/gradients_impl.py:108: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
2019-05-30 20:29:05,275	INFO policy_evaluator.py:728 -- Built policy map: {'default_policy': <ray.rllib.agents.ppo.ppo_policy_graph.PPOPolicyGraph object at 0x7fb9040c1c18>}
2019-05-30 20:29:05,276	INFO policy_evaluator.py:729 -- Built preprocessor map: {'default_policy': <ray.rllib.models.preprocessors.NoPreprocessor object at 0x7fb90

In [6]:
with open('best.wgt', 'rb') as f:
    weights = pickle.load(f)
agent.set_weights(weights) # set weights to local evaluator

In [7]:
writer = JsonWriter("./expert_sample")

In [8]:
for _ in range(5):
    samples = agent.sample(agent.train_batch_size)
    writer.write(sample_batch=samples)

(pid=353) Loading configuration... done.
(pid=338) Loading configuration... done.
(pid=338) Success.
(pid=367) Loading configuration... done.
(pid=367) Success.
(pid=353) Success.
(pid=338) Loading configuration... done.
(pid=367) Loading configuration... done.
(pid=353) Loading configuration... done.
(pid=338) 2019-05-30 20:29:23,095	INFO policy_evaluator.py:311 -- Creating policy evaluation worker 3 on CPU (please ignore any CUDA init errors)
(pid=338) 2019-05-30 20:29:23.096938: I tensorflow/core/platform/cpu_feature_guard.cc:141] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 AVX512F FMA
(pid=367) 2019-05-30 20:29:23,107	INFO policy_evaluator.py:311 -- Creating policy evaluation worker 1 on CPU (please ignore any CUDA init errors)
(pid=367) 2019-05-30 20:29:23.109556: I tensorflow/core/platform/cpu_feature_guard.cc:141] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 AVX512F FMA
(pid=353) 2019-05-30 

(pid=338) Loading configuration... done.
(pid=338) Success.
(pid=338) Loading configuration... done.
(pid=367) Loading configuration... done.
(pid=367) Success.
(pid=367) Loading configuration... done.
(pid=353) Loading configuration... done.
(pid=353) Success.
(pid=353) Loading configuration... done.
(pid=367) 2019-05-30 20:29:41,315	INFO policy_evaluator.py:474 -- Completed sample batch:
(pid=367) 
(pid=367) { 'data': { 'action_prob': np.ndarray((1779,), dtype=float32, min=0.002, max=1.536, mean=0.712),
(pid=367)             'actions': np.ndarray((1779, 1), dtype=float32, min=-2.857, max=2.261, mean=0.548),
(pid=367)             'advantages': np.ndarray((1779,), dtype=float32, min=0.905, max=35.978, mean=23.493),
(pid=367)             'agent_index': np.ndarray((1779,), dtype=int64, min=0.0, max=7.0, mean=3.445),
(pid=367)             'behaviour_logits': np.ndarray((1779, 2), dtype=float32, min=-1.352, max=1.358, mean=-0.176),
(pid=367)             'dones': np.ndarray((1779,), dtype=b

2019-05-30 20:29:42,467	INFO json_writer.py:97 -- Writing to new output file <_io.TextIOWrapper name='/headless/rl_project/flow_codes/InverseRL/expert_sample/output-2019-05-30_20-29-42_worker-0_0.json' mode='w' encoding='UTF-8'>


(pid=353) Loading configuration... done.
(pid=353) Success.
(pid=353) Loading configuration... done.
(pid=338) Loading configuration... done.
(pid=338) Success.
(pid=338) Loading configuration... done.
(pid=367) Loading configuration... done.
(pid=367) Success.
(pid=367) Loading configuration... done.
(pid=353) Warning: Teleporting vehicle 'flow_0.34'; collision with vehicle 'flow_2.3', lane=':center_1_0', gap=-1.00, time=126.20 stage=move.
(pid=353) Warning: Vehicle 'flow_0.34' ends teleporting on edge 'center', time 126.20.
(pid=353) Loading configuration... done.
(pid=353) Success.
(pid=353) Loading configuration... done.
(pid=338) Loading configuration... done.
(pid=338) Success.
(pid=338) Loading configuration... done.
(pid=367) Loading configuration... done.
(pid=367) Success.
(pid=367) Loading configuration... done.
(pid=353) Loading configuration... done.
(pid=353) Success.
(pid=353) Loading configuration... done.
(pid=367) Loading configuration... done.
(pid=367) Success.
(pid

In [12]:
reader = JsonReader("./expert_sample")

2019-05-30 20:31:25,864	WARNING json_reader.py:52 -- Treating input directory as glob pattern: /headless/rl_project/flow_codes/InverseRL/expert_sample/*.json
2019-05-30 20:31:25,866	INFO json_reader.py:65 -- Found 1 input files.


In [23]:
sample = reader.next()
sample.count

5247

In [15]:
sample.count

5247

In [18]:
samples.slice(0, sample.count).count

5247